# SLT-CE-1: Locally Linear Embedding

<h2 style="background-color:#f0b375;"> Setup </h2>

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn as skl
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
from sklearn.utils.validation import check_is_fitted

from sklearn.manifold import LocallyLinearEmbedding as SklearnLLE

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

<p style="background-color:#adebad;">
The MNIST data set contains three sets: **mnist.train, mnist.validation, mnist.test**  
Each of them is a numpy array with samples along rows, and pixels along columns.   
The original shape of the images is 28x28 = 784.
</p>

In [ ]:
mnist.train.images.shape

In [ ]:
mnist.validation.images.shape

In [ ]:
mnist.test.images.shape

In [ ]:
plt.matshow(mnist.train.images[2].reshape(28,28))
plt.show()

<h2 style="background-color:#f0b375;">
Section 4.0 
<span style=font-size:50%> Complete all problems in this section to get a pass on this exercise. </span>
</h2>

<p style="background-color:#adebad;">
    Shortly recapitulate the Locally Linear Embedding (LLE) algorithm, and the involved formulas.
</p>

Idea of the LLE algorithm is to learn a non-linear embedding by approximating it with local linear patches. At each point, we do a different linear embedding as the manifold looks linear locally. Implemetation is done accordingly to the [pseudocode](https://cs.nyu.edu/~roweis/lle/algorithm.html) provided by the authors of the original paper.

Algorithmically, we iterate over the data points and find neighbors for each point. Using k-nearest neighbors to obtain the local patch gives the advantage of defining the patches adaptively with respect to sample distribution density. There are alternatives to allocate the local patch, e.g. using an epsilon-ball.

We solve for the weights that reconstruct the datapoint from its neighbors and set all non-neighbors weights to zero, for every point in the training data set. That is, we first define de local covariance as $ C=Z^T Z $, where $ Z $ is the matrix that consists of all the neighbors of $ X $, then solve for $ Cw = 1 $ and assign the normalized $ w $ values as weights of the neighbors. 

After computing weights at each data point we create the sparse matrix $ M = (I-W)^T(I-W) $ and get the bottom $d+1$ eigenvectors of it, discarding the bottom eigenvector with eigenvalue zero. This gives the embedding vectors of the original data points.

Over the course of the algorithm, we also calculate and save various variables, including the reconstruction error of the embedding and the eigenvalues of the matrix $ M $.

Resulting embedding vectors are plotted and labeled alongside the ones obtained from the scikit-learn's implementation.

<p style="background-color:#adebad;">
    Implement the <b>fit method</b> for the template class LocallyLinearEmbedding, according to the contract outlined in its docstring.<br>
    In particular, use the class attribute (variable) names as provided, and introduce new class attributes only if necessary.
</p>

In [ ]:
class LocallyLinearEmbedding(skl.base.BaseEstimator, skl.base.TransformerMixin):
    """Template class for LLE, compare to `sklearn LLE`_.
    
    Attributes:
        embedding_vectors_ (np.ndarray): Embedding of input X with shape (samples, n_components)
        nbrs_X (e.g. sklearn.neighbors.KDTree): NearestNeighbors (NN) object, stores NN for training set X.
        nbrs_y (e.g. sklearn.neighbors.KDTree): NearestNeighbors (NN) object, stores NN for embedding_vectors_.
        M_ (np.array): Symmetric matrix (samples, samples), used in quadratic form for embedding.
        X_ (np.array): Copy of training set with shape (samples, dim).
    
       .. _`sklearn LLE`: http://scikit-learn.org/stable/modules/generated/sklearn.manifold.LocallyLinearEmbedding.html
    """
    
    def __init__(self, n_components=2, n_neighbors=5):
        self.n_neighbors = n_neighbors
        self.n_components = n_components
    
    def fit(self, X):
        """Compute LLE embedding of vectors X
        
        First, compute nbrs_X and M_.
        Finally, compute embedding_vectors_.
        
        Args:
            X (np.ndarray): Input array with shape (samples, dim)
        
        Returns:
            self
        """
        self.X_ = X
        self.nbrs_X = skl.neighbors.KDTree(self.X_, leaf_size=100)
        
        n_items = X.shape[0]
        weights = np.zeros((n_items, self.X_.shape[0]))

        for i in range(0, n_items):
            
            # Current datapoint
            cur = X[i,:]
            
            # Find the nearest neighbors
            '''
            dists = (X - cur)
            dists = np.multiply(dists, dists)
            dists = np.sum(dists, axis=1)
            sorted_dists = np.argsort(dists) # only need the first nn+1 elements, no need to sort it all
            n_idxs = sorted_dists[1:self.n_neighbors+1]
            print(n_idxs)
            '''
            # Instead, use the sklearn implementation
            dist, ind = self.nbrs_X.query(cur.reshape(1, -1), k=self.n_neighbors+1)
            n_idxs = np.setdiff1d(ind[0],[i])
            neighbors = self.X_[n_idxs, :]
            
            # Find the weights
            # Implementation 1
            '''
            cur_weights = np.linalg.lstsq(np.transpose(neighbors), cur, rcond=None)
            cur_weights = cur_weights[0] / np.sum(cur_weights[0])
            weights[i, n_idxs] = cur_weights
            '''
            # Implementation 2
            '''
            matrix_c = np.zeros((self.n_neighbors, self.n_neighbors))
            for j in range(0,self.n_neighbors):
                for k in range(0,self.n_neighbors):
                    matrix_c[j,k] = np.dot(np.transpose(cur - self.X_[n_idxs[j], :]), (cur - self.X_[n_idxs[k], :]))
            c_inv = np.linalg.inv(matrix_c)
            sum_c_inv = np.sum(c_inv)
            for j in range(self.n_neighbors):
                weights[i,n_idxs[j]] = np.sum(c_inv[j, :]) / sum_c_inv
            '''
            # Implementation 3
            dists = cur - neighbors
            c_inv = np.linalg.inv(np.matmul(dists, np.transpose(dists)))
            sum_c_inv = np.sum(c_inv)
            for j in range(self.n_neighbors):
                weights[i,n_idxs[j]] = np.sum(c_inv[j, :]) / sum_c_inv

            # Sanity check for the found weights
            if(False):
                sanity = np.matmul(weights[i, :], X)
                plt.matshow(cur.reshape(28,28))
                plt.show()
                plt.matshow(sanity.reshape(28,28))
                plt.show()

        self.weights = weights
        #print('Finished calculating weights')
        
        # Implementation 1: Compute the embeddings and the reconstruction error
        matrix_i = np.identity(n_items)
        i_w = matrix_i - weights
        matrix_m = np.matmul(np.transpose(i_w), i_w)
        
        # Get the eigenvectors using svd
        '''
        u, s, vh = np.linalg.svd(matrix_m)
        y = u[:,-(self.n_components+1):-1]
        self.recon_error = np.sum(s[-(self.n_components+1):-1])
        '''
        
        # Implementation 2: Get the eigenvectors using eigh
        s, u = np.linalg.eigh(matrix_m)
        #arg_s = np.flip(np.argsort(s))
        #v = u[:,arg_s]
        y = u[:,1:(self.n_components+1)]
        self.recon_error = np.sum(s[1:(self.n_components+1)])
        self.m_eigvals = s
        
        # Alter the columns so we can compare the implementation to that of sk-learn
        #y = y[:,::-1]
        #y[:,:] = -y[:,:]
        y[:,0] = -y[:,0]
        
        # Set the class variables
        self.embedding_vectors_ = y
        self.M_ = matrix_m
        self.nbrs_y = skl.neighbors.KDTree(self.embedding_vectors_, leaf_size=100)
    
    def transform(self, X):
        """Map new vectors X to embedding space
        
        Use the fitted model to map new vectors to the space with dimension n_components.
        
        Args:
            X (np.ndarray): Input array with shape (new_samples, dim)
            
        Returns:
            y (np.ndarray): Embedded vectors with shape (new_samples, n_components)
        """

        check_is_fitted(self, ["embedding_vectors_", "nbrs_X"])

        n_items = X.shape[0]
        weights = np.zeros((n_items, self.X_.shape[0]))
        for i in range(0, n_items):
            
            # Current datapoint
            cur = X[i,:]
            
            # Find the nearest neighbors
            dist, ind = self.nbrs_X.query(cur.reshape(1, -1), k=self.n_neighbors+1)
            n_idxs = np.setdiff1d(ind[0],[i])
            neighbors = self.X_[n_idxs, :]
            
            # Find the weights
            cur_weights = np.linalg.lstsq(np.transpose(neighbors), cur, rcond=-1)
            cur_weights = cur_weights[0] / np.sum(cur_weights[0])
            weights[i, n_idxs] = cur_weights

        #print('Finished calculating weights')
        
        # Find the embeddings
        y = np.matmul(weights, self.embedding_vectors_)
        return y

    def inverse_transform(self, y):
        """Map new vectors y to input space with dimension dim.
        
        Use the fitted model to map vectors y to the original input space.
        
        Args:
            y (np.ndarray): Input array with shape (new_samples, n_components)
            
        Returns:
            X (np.ndarray): Vectors with shape (new_samples, dim)
        """

        check_is_fitted(self, ["embedding_vectors_", "X_", "nbrs_y"])
 
        n_items = y.shape[0]
        weights = np.zeros((n_items, self.embedding_vectors_.shape[0]))

        for i in range(0, n_items):
            
            # Current datapoint
            cur = y[i,:]

            # Find the neighbors
            dist, ind = self.nbrs_y.query(cur.reshape(1, -1), k=self.n_neighbors+1)
            n_idxs = np.setdiff1d(ind[0],[i])
            neighbors = self.embedding_vectors_[n_idxs, :]
            
            # Find the weights
            cur_weights = np.linalg.lstsq(np.transpose(neighbors), cur, rcond=-1)
            cur_weights = cur_weights[0] / np.sum(cur_weights[0])
            weights[i, n_idxs] = cur_weights

        self.weights = weights
        #print('Finished calculating weights')
        
        # Find the inverse transforms
        X = np.matmul(weights, self.X_)
        return X


<p style="background-color:#adebad;">
    Create an instance of your LLE class with default parameters and fit the MNIST validation set. Record the execution time (should be about 30 sec). Furthermore, create an instance of the sklearn LLE class, and fit it with the same parameters. Again record the execution time.  
    Note, the dots should be filled in by you.
</p>

In [ ]:
%%time
myLLE = LocallyLinearEmbedding()
myLLE.fit(mnist.validation.images)

In [ ]:
%%time
sklLLE = SklearnLLE(random_state=42)
sklLLE.fit(mnist.validation.images)

<p style="background-color:#adebad;">
    Plot myLLE.embedding_vectors_ and sklLLE.embedding_vectors_ in two separate plots.
</p>

In [ ]:
def plt_mnist(embedding, of="validation"):
    labels = getattr(mnist, of).labels

    if len(labels) != len(embedding):
        raise ValueError("Number of labels does not match number of embeddings")
    
    for label in range(10):
        idx = (labels == label)
        plt.scatter(embedding[idx, 0], embedding[idx, 1], alpha=0.5, label=str(label))
    plt.legend()

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(121)
plt_mnist(sklLLE.embedding_)
plt.subplot(122)
plt_mnist(myLLE.embedding_vectors_)

In [ ]:
print('Fit - Errors w.r.t. sklearn implementation: ')
print('Avg. error:', np.average(np.abs(sklLLE.embedding_ - myLLE.embedding_vectors_)))
print('Max. erorr:', np.max(np.abs(sklLLE.embedding_ - myLLE.embedding_vectors_)))

<h2 style="background-color:#f0b375;"> Section 4.5
<span style=font-size:50%> Complete all problems in this and previous sections to get a 4.5 on this exercise. </span>
</h2>

<p style="background-color:#adebad;">
    <ul style="background-color:#adebad;">
        <li> Implement the <b>transform method</b> for the template class LocallyLinearEmbedding, according to the contract outlined in its docstring.
        </li>
        -
        <li> Fit both, myLLE and sklLLE on the MNIST validation set, and use the fitted model to transform the MNIST test set.
        </li>
        -
        <li>
            Plot both embeddings in two separate plots for comparison.
        </li>
    </ul>
</p>

Idea behind the **transform** method is quite straightforward, having implemented the **fit** method. We look for the neighbors of the new (unseen) data point in the training data set and solve for its weights. This time implementation uses a least square solver to solve the linear system. Then, we use the calculated weights on the original embedding vectors to obtain the embedding of the new data point, hoping the local approximation would be satisfactory. User can again compare the results of the implementation provided here against the scikit-learn's implementation.

In [ ]:
%%time
my_embedding = myLLE.transform(mnist.test.images)

In [ ]:
%%time
skl_embedding = sklLLE.transform(mnist.test.images)

In [ ]:
# Put the plots here
plt.figure(figsize=(15,10))
plt.subplot(121)
plt_mnist(skl_embedding, of='test')
plt.subplot(122)
plt_mnist(my_embedding, of='test')

In [ ]:
print('Transform - Errors w.r.t. sklearn implementation: ')
print('Avg. error:', np.average(np.abs(skl_embedding - my_embedding)))
print('Max. erorr:', np.max(np.abs(skl_embedding - my_embedding)))

<h2 style="background-color:#f0b375;"> Section 5.0
<span style=font-size:50%> Complete all problems in this and previous sections to get a 5.0 on this exercise. </span>
</h2>

<ul style="background-color:#adebad;">
    <li> Think how you can invert the LLE embedding, and shortly describe your approach.</li>
        <li> Using your approach, implement the <b>inverse_transform method</b> for the template class LocallyLinearEmbedding, according to the contract outlined in its docstring.
        </li>
        <li>
        Use myLLE.transform and myLLE.inverse_transform to embed the first digit of the MNIST training set and then recover it. 
        </li>
       
   </ul>

**inverse_transform** method implements a strategy quite similar to that we followed through in **transform** method. This time, we want to obtain the data point in the original manifold, given its embedding in the d-dimensional space we have constructed in **fit**. We find the neighboring embeddings for a given embedding, obtain the reconstruction weights using a least square solver, then use these weights to approximate the data point in the original manifold using the weighted sum of the original data points that correspond the neighboring embeddings. It is the same strategy in **transform**, as we have indicated earlier, using embeddings to transform a new embedding into the original manifold instead of vice-versa. Authors also argue that regularisation could be of help when there are more neighbors than input dimensions ($k \geq d$).

In [ ]:
digit_idx = 1
embedded_digit = myLLE.transform(np.asarray([mnist.train.images[digit_idx]]))
reconstructed_digit = myLLE.inverse_transform(embedded_digit)

<ul style="background-color:#adebad;">
    <li>
        Plot the original digit and the reconstructed digit for comparison.
    </li>
</ul>

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(121)
plt.imshow(mnist.train.images[digit_idx].reshape(28,28))
plt.subplot(122)
plt.imshow(reconstructed_digit[0].reshape(28,28))

<h2 style="background-color:#f0b375;"> Section 5.5
<span style=font-size:50%> Complete all problems in this and previous sections to get a 5.5 on this exercise. </span>
</h2>

<ul style="background-color:#adebad;">
        <li>
        Perform a grid search of the reconstruction error, i.e. the sum of the k lowest eigenvalues of matrix M_ (with k=n_components), over the parameters n_neighbors, and n_components.  
        Create a nice 2D heatmap of the results.
        </li>
    </ul>

In [ ]:
list_neighbors = [3,4,5,7,10]
list_components = [2,4,6,8,10]
my_errormap = np.zeros((len(list_neighbors), len(list_components)))
for i, nn in enumerate(list_neighbors):
    for j, nc in enumerate(list_components):
        # Fit the embeddings
        myLLE = LocallyLinearEmbedding(n_components=nc, n_neighbors=nn)
        myLLE.fit(mnist.validation.images)
        my_errormap[i,j] = myLLE.recon_error
        print(nn, nc, myLLE.recon_error)
        '''
        # Sanity check: calculate the errors
        YtMY = np.matmul(np.transpose(myLLE.embedding_vectors_), np.matmul(myLLE.M_, myLLE.embedding_vectors_))
        u, s, vh = np.linalg.svd(myLLE.M_)
        YtMY_error = np.trace(YtMY)
        trm_error = np.sum(s[-(nc+1):-1])
        fro_error = np.linalg.norm(myLLE.embedding_vectors_- np.matmul(myLLE.weights, myLLE.embedding_vectors_), 'fro')**2
        print(myLLE.recon_error, YtMY_error, trm_error, fro_error)
        ''' 

Reconstrution error increases with both n_components and n_neighbors. This is natural as we add entries to the sum by increasing both values.

In [ ]:
plt.imshow(my_errormap, cmap='hot')
plt.xticks( range(len(list_components)), list_components )
plt.xlabel('n_components')
plt.yticks( range(len(list_neighbors)), list_neighbors )
plt.ylabel('n_neighbors')
plt.show()

<ul style="background-color:#adebad;">
        <li>
        Create a matrix plot of the matrix M_.  
        Make sure to permute the rows and columns according to the digit labels, so that you can actually observe the block structure. Also make sure to scale the plot properly, so that you can observe the block structure. What is the origin of the block structure?
        </li>
        <li>
        Plot the spectrum (eigenvalues) of M_. Can you identify a good cutoff? What could you use the value of the cutoff for?
        </li>
    </ul>

We run the fit method one more time to obtain results that are comparable in the sections preceding the grid search, i.e. we would again be using default parameters for the algorithm in the following sections.

In [ ]:
myLLE = LocallyLinearEmbedding()
myLLE.fit(mnist.validation.images)

Rows and columns are ordered accordingly to the digit labels. White pixels denote the non-zero entries whereas the black pixels denote the zeroes. Origin of the block structure is the labels having neighbors moslty from their own labels, as non-zero entries are neighbor weights.

We can see that 0 having a denser block than rest as intraclass variation w.r.t. shape is small for 0s, making the digits clustered in the embedding space. Similarly, block for 5 is perhaps the most sparse block in the matrix, indicating that the digits being most distorted by the transformation. Additionally, 4s or 8s only having neighbors (aside from their own labels) with 9s is intuitive by the similarity of the shapes.

In [ ]:
labels = getattr(mnist, 'validation').labels
arg_lab = np.argsort(labels)
matrix_m = myLLE.M_
matrix_m[0] = np.inf
matrix_m = matrix_m[arg_lab,:]
matrix_m = matrix_m[:, arg_lab]
plt.figure(figsize=(10,10))
plt.imshow(matrix_m, cmap='hot')
plt.show()

Below is illustrated the eigenvalues of the matrix M in ascending order. Looking at the zoomed in chart on the right side, 8 eigenvalues seems to be a good cutoff, although one can argue on different cutoffs.

In [ ]:
eigvals = myLLE.m_eigvals
plt.figure(figsize=(16,4))
plt.subplot(121)
plt.plot(eigvals)
plt.xlabel('Spectrum index')
plt.ylabel('Eigenvalue')
plt.grid()
plt.subplot(122)
plt.plot(eigvals[:20])
plt.xticks(range(0,len(eigvals[:20])))
plt.xlabel('Spectrum index')
plt.ylabel('Eigenvalue')
plt.grid()
plt.show()

<h2 style="background-color:#f0b375;"> Section 6.0
<span style=font-size:50%> Complete all problems in this and previous sections to get a 6.0 on this exercise. </span>
</h2>

<ul style="background-color:#adebad;">
        <li>
        Plot the linear interpolation between two digits $x_1,\ x_2$ in the input space compared to the reconstruction along the linear interpolation of their embeddings $y_1, y_2$. More precisely, compare $\lambda x_1 + (1-\lambda) x_2,\ \lambda\in[0,1]$ to $LLE^{-1}\big(\lambda y_1 + (1-\lambda) y_2\big),\ \lambda\in[0,1]$
        </li>
    </ul>

**plt_with_bg plots** a trajectory along a series of embeddings on top of provided background embeddings with class labels

In [ ]:
def plt_with_bg(fg_embedding, fg_tag, bg_embedding, of="validation"):
    labels = getattr(mnist, of).labels
    if len(labels) != len(bg_embedding):
        raise ValueError("Number of labels does not match number of embeddings")
    for label in range(10):
        idx = (labels == label)
        xs = bg_embedding[idx, 0]
        ys = bg_embedding[idx, 1]
        plt.scatter(xs[::], ys[::], alpha=0.2, label=str(label))

    plt.plot(fg_embedding[:, 0], fg_embedding[:, 1], alpha=0.9, label=fg_tag, marker='o',
             markerfacecolor='w', markeredgewidth=1.5, markeredgecolor='k')
    plt.legend()

**plt_interpolate plots** the interpolation of two images and the inverse transform of their interpolated embeddings

In [ ]:
def plt_interpolate(d1, d2):
    embed_list = []
    # Set the discretization level
    k = 15
    # Plot the linear interpolation
    plt.figure(figsize=(15,1))
    weight = np.linspace(0, 1, num=k)
    for i in range(0,k):
        plt.subplot(1, k, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow((d2*weight[i] + d1*(1-weight[i])).reshape(28,28), cmap=plt.cm.gray)
    plt.show()
    # Plot the linear interpolation of their embeddings
    plt.figure(figsize=(15,1))
    weight = np.linspace(0, 1, num=k)
    for i in range(0,k):
        plt.subplot(1, k, i+1)
        plt.xticks([])
        plt.yticks([])
        embedded = myLLE.transform(np.asarray([d2]))*weight[i] + myLLE.transform(np.asarray([d1]))*(1-weight[i])
        plt.imshow(( myLLE.inverse_transform(embedded)[0] ).reshape(28,28), cmap=plt.cm.gray)
        embed_list.append(embedded)
    plt.show()
    embed_list = np.concatenate(embed_list, axis=0)
    return embed_list

Interpolation of the two images of the same digit

In [ ]:
d1 = mnist.train.images[1]
d2 = mnist.train.images[11]
embed_list = plt_interpolate(d1, d2)
plt.figure(figsize=(6,6))
plt_with_bg(embed_list, 'ip3-3', myLLE.embedding_vectors_, of="validation")
plt.show()

Interpolation of images of different digits

In [ ]:
d1 = mnist.train.images[1]
d2 = mnist.train.images[2]
embed_list = plt_interpolate(d1, d2)
plt.figure(figsize=(6,6))
plt_with_bg(embed_list, 'ip3-4', myLLE.embedding_vectors_, of="validation")
plt.show()

<ul style="background-color:#adebad;">
        <li>
        Select images of digits "6" and a digits "8".  
        Rotate the input images by 360° in steps of 1° and create the embedding of each rotation.  
        Create a nice plot of the respective trajectories in the 2D embedding space (Make sure to use markers for the embeddings, and connect them with a line. Use labels and legends, and add some embeddings of other digits to put the trajectories into context).
        </li>
    </ul>

Function anim.to_html5_video() may not work if the ffmpeg codec (or any alternative one) is not installed properly.
Similarly, anim.to_jshtml() might not work as it requires considerable memory to create interactive plots.

If for any other reason the animations do not work, user can view the static plots below the animated ones.

In [ ]:
%matplotlib inline
from scipy import ndimage
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

**rotate_and_embed** rotates the given image 360° in steps of 1°, then creates and returns the embeddings of the rotated images

In [ ]:
def rotate_and_embed(img,angle_step):
    rot_list = []
    org_size = img.shape
    for degree in range(0,360,angle_step):
        rotated_img = ndimage.rotate(img, angle=degree, order=1, reshape=False, axes=(0,1))
        '''
        # Alternative implementation
        rotated_img = ndimage.rotate(img, degree)
        size_diff = np.subtract(rotated_img.shape, org_size)[0]
        i = size_diff//2
        j = size_diff - size_diff//2
        rlen = rotated_img.shape[0]
        rotated_img = rotated_img[i:rlen-j,i:rlen-j]   
        '''
        rot_list.append(rotated_img.flatten())
        if(False):
            print(degree, rotated_img.shape, size_diff)
            plt.figure()
            plt.imshow(rotated_img, cmap=plt.cm.gray)
            plt.show()
    res_list = np.stack(rot_list, axis=0)
    res_list = myLLE.transform(res_list)

    return rot_list, res_list

**plt_rotate_animate** creates both animations for rotating image itself and the trajectory of its embedding

In [ ]:
def plt_rotate_animate(digit, tag):
    fig, ax = plt.subplots(1, 2)
    fig.set_size_inches(12, 10, True)
    ax[0].set_xlim((-0.02, 0.04))
    ax[0].set_ylim((-0.05, 0.04))

    labels = getattr(mnist, 'validation').labels
    for label in range(10):
        idx = (labels == label)
        xs = myLLE.embedding_vectors_[idx, 0]
        ys = myLLE.embedding_vectors_[idx, 1]
        ax[0].scatter(xs, ys, alpha=0.2, label=str(label))

    xdata, ydata = [], []
    line, = ax[0].plot([], [], alpha=1, label=tag, marker='o', markerfacecolor='w', markeredgewidth=1.5, markeredgecolor='k')
    ax[0].legend()

    def init():
        line.set_data([], [])
        return (line,)
    def animate(i):
        xdata.append(rot_embedding[i, 0])
        ydata.append(rot_embedding[i, 1])
        line.set_data(xdata, ydata)
        global x, y
        x += np.pi / 15.
        y += np.pi / 20.
        im.set_array(f(x, y))
        im.set_array(rot_imgs[i].reshape(28,28))
        return (line,im,)

    def f(x, y):
        return np.sin(x) + np.cos(y)

    x = np.linspace(0, 2 * np.pi, 120)
    y = np.linspace(0, 2 * np.pi, 100).reshape(-1, 1)

    im = ax[1].imshow(f(x, y), animated=True)

    rot_imgs, rot_embedding = rotate_and_embed(digit, 1)
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=360, interval=100, blit=True)

    #HTML(anim.to_html5_video())
    #HTML(anim.to_jshtml())
    
    return anim

Animation showing the movement in the embedding space corresponding the rotating image given to its right. This example uses an image of the digit 6 . It is in line with our expectations that the upside down image should be embedded near 9 s in the embedding space. It stays there for a considerable amount and returns to its original place as the image nears to complete its rotation. The intermediary movements are harder to explain intuitively as they do not quite look like any of the digits in the training data set the movement is naturally erratic.

In [ ]:
six = mnist.train.images[3].reshape(28,28)
anim = plt_rotate_animate(six, 'rot6')
HTML(anim.to_html5_video())

The situation here with an exemplary image of 8 is similar to what we have explained above, but even more irregular. We can visually confirm the embedding return near the original position when the rotation angle is around 180°, as well as when it is aroudn 360°. In the intermediary steps, its movement is more unpredictable than in the former example as we now deal with a more complicated shape in the original manifold.

In [ ]:
eight = mnist.train.images[5].reshape(28,28)
anim = plt_rotate_animate(eight, 'rot8')
HTML(anim.to_html5_video())

Please run the cell below for the static plots, if the animated plots do not display properly.

In [ ]:
six = mnist.train.images[3].reshape(28,28)
eight = mnist.train.images[5].reshape(28,28)
plt.figure(figsize=(15,10))
plt.subplot(121)
plt_with_bg(rotate_and_embed(six, 1)[1], 'rot6', myLLE.embedding_vectors_, of='validation')
plt.subplot(122)
plt_with_bg(rotate_and_embed(eight, 1)[1], 'rot8', myLLE.embedding_vectors_, of='validation')
plt.show()